In [79]:
%load_ext autoreload
%autoreload 2
import torch
from augment import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
space = AugmentationSpace()

In [81]:
test_data = torch.rand((10, 3, 32, 32))

In [82]:
policy = [
    [  # Sub-policy 1
        {'operation': 5, 'magnitude': 3, 'probability': 0.7},  # Cutout at 70%
        {'operation': 3, 'magnitude': 2, 'probability': 0.8}   # Shear_x at 80%
    ],
    [  # Sub-policy 2
        {'operation': 1, 'magnitude': 4, 'probability': 0.3},  # Translate_x at 30%
        {'operation': 3, 'magnitude': 1, 'probability': 0.9}   # Brightness at 90%
    ]
]

In [83]:
augmented_data = apply_auto_augmentations(test_data, policy, space)
augmented_data.shape

torch.Size([10, 3, 32, 32])

In [85]:
augmented_data = apply_random_augmentations(test_data, space)
augmented_data.shape

torch.Size([10, 3, 32, 32])